In [1]:
import os
import stat
import time
from tqdm import tqdm
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document, StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.vectorstores import Chroma
from colorama import Fore
import warnings

warnings.filterwarnings("ignore")

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LANGUAGE_MODEL = "gpt-4 turbo"
llm = ChatOpenAI()


In [2]:
#### ارزیاب بازیابی : ارزیاب ارتباط ####
class GradeDocuments(BaseModel):
    """امتیاز باینری برای ارزیابی ارتباط اسناد بازیابی‌شده."""

    binary_score: str = Field(description="اسناد نسبت به پرسش کاربر مرتبط هستند: 'بله' یا 'خیر'")

    def get_score(self) -> str:
        """امتیاز باینری را به صورت رشته برمی‌گرداند."""
        return self.binary_score


def get_score(self) -> str:
    """امتیاز باینری را به صورت رشته برمی‌گرداند."""
    return self.binary_score

# استفاده از LLM با فراخوانی تابع ساختارمند
structured_llm_grader = llm.with_structured_output(GradeDocuments)


In [3]:
# Prompt 

system_template = """تو یک ارزیاب هستی که ارتباط اسناد بازیابی شده زیر:/
{documents}/
با پرسش کاربر:/
{question}/
را تعیین می کنید. اگر سند شامل کلمه های کلیدی با معانی مرتبط با پرسش باشد آنرا به عنوان مرتبط علامتگداری کرده و پاسخ:
بله/
و اگر نه پاسخ:
خیر/
را انتخاب کنید./
سعی کن در پاسخ خود سخت گیر باشی و اگر اطمینان نداری که سند مرتبط است پاسخ خیر را انتخاب کنی./"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["documents", "question"],
    template="{question}",
)
grader_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

In [4]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


def load_documents():
    """
    Load all text files from a directory, split them into chunks,
    and add metadata with 'doc_id' and 'chunk_index' for each chunk.
    """
    loader = DirectoryLoader("./output_text/", glob="*.txt")  # Load all .txt files
    raw_documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=100,
        chunk_overlap=0,
        separators=["\n\n", "\n", " ", ""]
    )
    
    all_chunks = []
    for raw_doc in raw_documents:
        # Get a document identifier. Here we use the 'source' metadata if available.
        doc_id = raw_doc.metadata.get("source", "unknown")
        chunks = text_splitter.split_text(raw_doc.page_content)
        for idx, chunk in enumerate(chunks):
            new_doc = Document(page_content=chunk, metadata={"doc_id": doc_id, "chunk_index": idx})
            all_chunks.append(new_doc)
    return all_chunks


def load_embeddings(documents, user_query):
    """
    Create or load a Chroma vector store from a set of documents.
    """
    persist_directory = './chroma_cache'  # Directory to store embeddings
    embedding_model = OpenAIEmbeddings()

    # Ensure the directory exists and has write permissions
    if not os.path.exists(persist_directory):
        os.makedirs(persist_directory, exist_ok=True)
    else:
        if not os.access(persist_directory, os.W_OK):
            print(f"Error: No write access to {persist_directory}. Fixing permissions...")
            try:
                os.chmod(persist_directory, stat.S_IWUSR | stat.S_IRUSR | stat.S_IXUSR)
            except Exception as e:
                print(f"Failed to change directory permissions: {e}")
                return None

    try:
        # Load or create Chroma vector store
        if not os.listdir(persist_directory):  # Empty directory means no existing DB
            print("Initializing new ChromaDB instance...")
            db = Chroma.from_documents(documents, embedding_model, persist_directory=persist_directory)
            db.persist()
        else:
            print("Loading existing ChromaDB instance...")
            db = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)

        # For debugging: perform a similarity search with score and print the top result
        docs_with_scores = db.similarity_search_with_score(user_query, k=1)
        if docs_with_scores:
            top_doc, score = docs_with_scores[0]
            print("\nRetrieved Document (for debugging):\n")
            print(format_docs([top_doc]))
            print("\nSimilarity Score:", score)
        else:
            print("No documents retrieved for the query.")

        return db.as_retriever()

    except Exception as e:
        print(f"Error while loading ChromaDB: {e}")
        return None

In [5]:
def top_chunk(retriever , query):

    retrieved_chunks = retriever.get_relevant_documents(query)
    if not retrieved_chunks:
        return "No relevant document found."

    # Retrieve the top (most relevant) chunk and extract doc_id
    top_chunk = retrieved_chunks[0]

    return top_chunk.page_content

In [6]:
# Global documents for dynamic neighbor retrieval
documents = load_documents()

In [16]:
query_text_1 = """تفسیر بیت زیر چیست: /"
"یاد من کن پیش تخت آن عزیز /
تا مرا هم واخرد زین حبس نیز"""

query_text_2 = """تفسیر بیت زیر چیست: /

ای حیات دل حسام‌الدین بسی/
میل می‌جوشد به قسم سادسی"""

query_text_3 = """تفسیر بیت زیر چیست: /
گشت از جذب چو تو علامه‌ای/
در جهان گردان حسامی نامه‌ای"""

query_text_4 = """تفسیر بیت زیر چیست: /
"عارفی پرسید از آن پیر کشیش/
که توی خواجه مسن‌تر یا که ریش
"""

query_text_5 = """تفسیر بیت زیر چیست: /
خواجه‌ای را بود هندو بنده‌ای/
پروریده کرده او را زنده‌ای"""

query_text = query_text_2

retriever = load_embeddings(documents, query_text)

Loading existing ChromaDB instance...

Retrieved Document (for debugging):

از بیالفتی است، دل که دل بردید کی ماند ترش، بلبلی گل دید، کی ماند خمشماهی بریانز آسیب خرزر زنده شد،

Similarity Score: 0.24009285867214203


In [17]:
context = top_chunk(retriever, query_text)

print(context)

از بیالفتی است، دل که دل بردید کی ماند ترش، بلبلی گل دید، کی ماند خمشماهی بریانز آسیب خرزر زنده شد،


In [18]:
def assess_retrieve_docs(query, context):

    retrieval_grader = grader_prompt | structured_llm_grader | get_score
    binary_score = retrieval_grader.invoke({"question": query, "documents": context})
    
    return binary_score 

In [19]:
binary_score = assess_retrieve_docs(query_text, context)
print("binary score:", binary_score)

binary score: خیر


In [11]:
template: str = """/
    فرض کن تو یک مفسر مثنوی هستی. جواب سوال زیر را بده: /
      {question} /
   از محتوای زیر برای پیدا کردن مفاهیم و زمینه مربوطه استفاده کن. محتوای زیر از جلسات تفسیر مثنوی معنوی عبدالکریم سروش گرفته شده است./
      {context} /
      این محتوا از جلسه شماره زیر گرفته شده است:/
      {doc_name}/
       . سعی کن از این محتوا برای فهمیدن داستان و تاریخ مربوطه و ابیات مجاور بیت مورد سوال استفاده کنی./
       در پاسخی که میدهی سعی کن به زمینه داستانی و تاریخی و اشعار مجاور شعر مورد سوال در مثنوی اشاره کنی./
       در پاسخ خود به شماره جلسه ای که در مورد این شعر صحبت شده نیز اشاره کن./
       در پاسخ خود به تمام جزییاتی که از متن جلسه دریافت می کنی و مرتبط با سوال مطرح شده است اشاره کن.
    """

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["question", "context"],
    template="{question}",
)
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

### Question Re-writer - Knowledge Refinement ####
# Prompt 
prompt_template = """با در نظر گرفتن سوال زیر:/
{question},/
 دقت کن که سوال را به گونه ای بازنویسی کنی که مدل زبانی بتواند بهترین پاسخ را ارائه دهد./
 به سوال پاسخ نده. فقط سوال را بازنویسی کن./"""

system_prompt = SystemMessagePromptTemplate.from_template(prompt_template)
human_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["question"],
    template="{question}",
)
re_write_prompt = ChatPromptTemplate.from_messages(
    [system_prompt, human_prompt]
)

### Web Search Tool - Knowledge Searching ####
web_search_tool = TavilySearchResults(k=3) 


In [12]:
def rewrite_query(query):
    question_rewriter = re_write_prompt | llm | StrOutputParser()
    return question_rewriter.invoke({"question": query})

def search_web(query):
    docs = web_search_tool.invoke({"query": query})
    web_results = "\n".join([d["content"] for d in docs])
    return Document(page_content=web_results)
    
def generate_response(retriever, query):
    retrieved_chunks = retriever.get_relevant_documents(query)
    if not retrieved_chunks:
        return "No relevant document found."

    # Retrieve the top (most relevant) chunk and extract doc_id
    top_chunk = retrieved_chunks[0]
    doc_id = top_chunk.metadata.get("doc_id")
    chunk_index = top_chunk.metadata.get("chunk_index")

    # Find all chunks from the same document (using the global 'documents' variable)
    same_doc_chunks = [doc for doc in documents if doc.metadata.get("doc_id") == doc_id]
    same_doc_chunks = sorted(same_doc_chunks, key=lambda d: d.metadata.get("chunk_index", 0))

    # Define a window: e.g. 15 chunks before and after the top chunk
    start = max(0, chunk_index - 15)
    end = min(len(same_doc_chunks), chunk_index + 15)
    aggregated_context = "\n\n".join([doc.page_content for doc in same_doc_chunks[start:end]])

    # Build the chain and invoke it with the additional 'doc_name' variable
    chain = chat_prompt_template | llm | StrOutputParser()
    input_vars = {"context": aggregated_context, "question": query, "doc_name": doc_id}
    return chain.invoke(input_vars)

In [22]:
if binary_score == "خیر":
        print(f"{Fore.MAGENTA}Retrieval is not relevant. Searching the web...{Fore.RESET}")
        context = search_web(query_text) 

print(f"{Fore.YELLOW}Retrieval, rewriting and optmize the query...{Fore.RESET}")  
optimized_query = rewrite_query(query_text)

Retrieval is not relevant. Searching the web...
Retrieval, rewriting and optmize the query...


In [23]:
print(context)

page_content='001 ای حیات دل! حُسامالدین! بســی مِیل میجوشد به قسـمِ سـادسی. وقتی که میگوید حیات دل یعنی ای حسام الدینی که بخشنده حیات معنوی به دلهای مرده\nتفسیر مثنوی دفتر ۶ جلسهٔ ۱ بخش ۱ ۱ای حیات دل حسام\u200cالدین بسی میل می\u200cجوشد به قسم سادسی عموماً حُسام الدین ویا شمس کنایه ویا نمادی از منبع آگاهی\nدیباچه دفتر ششم مثنوی معنوی مولو ی | دکلمه ، شرح و تفسیر ، دفتر ششم مثنوی معنوی از مولانا جلال الدین محمد بلخی درمرکز شعر و عرفان وب سایت\nدر این کتاب ۴۲۴ داستان پی\u200cدرپی به شیوهٔ تمثیل داستان سختی\u200cهای انسان در راه رسیدن به خدا را بیان می\u200cکند. هجده بیت نخست دفتر اول مثنوی معنوی به نی\u200cنامه شهرت دارد و\nمولانا یا روشنگری خود این مشکل بزرگ هدایت و راهیابی در طول زمانها را حل می کند. میفرماید که سیب هدایت و آگاهی، بوی خوشی به سوی تو خواهد آورد،اما کلید هدایت آن'


In [24]:
response = generate_response(retriever, query_text)

print(response)

بیت فوق الذکر از مثنوی مولوی است که به این معنا است که "ای حیات، دل حسام‌الدین بسیار به سوی قسم سادسی میل و می‌جوشد". این بیت به دو شخصیت مهم در مثنوی اشاره دارد. 

در تفسیر این بیت، می‌توان به داستان حسام‌الدین و قسم سادسی اشاره کرد. حسام‌الدین، شخصیتی است که در مثنوی مولوی به عنوان یک عاشق بی‌قرار و بی‌نیاز به عالم مادی نمایان شده است. او به دنبال روحانیت و معنویت است و به دنبال عشق الهی می‌گردد. قسم سادسی به دیگر یکی از شخصیت‌های مثنوی اشاره دارد که نماد عشق و معشوق الهی است. حسام‌الدین به سوی این قسم سادسی میل می‌جوشد، یعنی عشق و اشتیاق او به عشق الهی بسیار بزرگ و بی‌پایان است.

در جلسه شماره ۶۱ دفتر ششم مثنوی، دکتر سروش در تفسیر این بیت به اشعار و مفاهیم مجاور اشاره کرده و روایت‌های مختلفی را برای شرح این بیت ارائه داده است. او به بررسی داستان حسام‌الدین و قسم سادسی پرداخته و تلاش کرده است تا عمق و غنای این بیت را برای شنوندگان روشن کند.

در نهایت، این بیت نشان‌دهنده عشق و اشتیاق عمیق حسام‌الدین به معشوق الهی است و نشان‌دهنده تمایل او به سوی ارتباط و اتحاد با خداوند می‌باشد.
